In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
mnist = tf.keras.datasets.mnist

(X_train,y_train),(X_test,y_test) = mnist.load_data()
X_train, X_test = X_train/255.0, X_test/255.0

#Add channel dimesnsion
X_train = X_train[...,tf.newaxis].astype("float32")
X_test = X_test[...,tf.newaxis].astype("float32")

In [3]:
X_train.shape

(60000, 28, 28, 1)

In [4]:
train_ds = tf.data.Dataset.from_tensor_slices((X_train,y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((X_test,y_test)).batch(32)

In [5]:
class MnistModel(Model):

  def __init__(self):

    super(MnistModel,self).__init__()
    self.conv1 = Conv2D(32,3,activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128,activation="relu")
    self.d2 = Dense(10)

  def call(self,x):

    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MnistModel()

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [7]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [9]:
## tf.GradientTape to train the model

@tf.function
def train_step(images,labels):

  with tf.GradientTape() as tape:

    predictions = model(images,training=True)
    loss = loss_fn(labels,predictions)

    gradients = tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(gradients,model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels,predictions)

In [10]:
@tf.function
def test_step(images, labels):
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [ ]:
epochs = 5

for epoch in range(epochs):

  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images,labels in train_ds:
    train_step(images,labels)
  
  for test_images in test_ds:
    test_step(images,labels)

  print (
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )